# Research

Research class is intended for multiple running of the same pipelines with different parameters in order to get some statistics.

Let's compare `VGG7` and `VGG16` performance on `MNIST` dataset with different layouts of convolutional blocks. For each combination of layout and model class, we will train model for 500 iterations and repeat that process 2 times. 

In [ ]:
import sys
import dill
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append("../..")
from dataset import Pipeline, B, C, V
from dataset.opensets import MNIST
from dataset.models.tf import VGG7, VGG16
from dataset.research import Research, Option

In [ ]:
BATCH_SIZE=64

# Define pipelines

Define model config. All parameters that we want to vary we define as ``C('parameter_name')``. In our case it's a `'body/block/layout'`.

In [ ]:
model_config={
    'session/config': tf.ConfigProto(allow_soft_placement=True),
    'inputs': dict(images={'shape': (28, 28, 1)},
                   labels={'classes': 10, 'transform': 'ohe', 'name': 'targets'}),
    'input_block/inputs': 'images',
    'body/block/layout': C('layout'),
    'output/ops': 'accuracy',
    'device': C('device') # it's technical parameter for TFModel
}

Define loading as a separate pipeline with lazy run. This is optional.

In [ ]:
mnist = MNIST()
train_root = mnist.train.p.run(BATCH_SIZE, shuffle=True, n_epochs=None, lazy=True)
test_root = mnist.test.p.run(BATCH_SIZE, shuffle=True, n_epochs=1, lazy=True)

Now define main parts of pipelines where we want to vary model:

In [ ]:
train_template = (Pipeline()
            .init_variable('loss', init_on_each_run=list)
            .init_variable('accuracy', init_on_each_run=list)
            .init_model('dynamic', C('model'), 'conv', config=model_config)
            .to_array()
            .train_model('conv', 
                         fetches=['loss', 'output_accuracy'], 
                         feed_dict={'images': B('images'), 'labels': B('labels')},
                         save_to=[V('loss'), V('accuracy')], mode='w')
)

In [ ]:
test_template = (Pipeline()
            .init_variable('accuracy', init_on_each_run=list)
            .import_model('conv', C('import_from'))
            .to_array()
            .predict_model('conv', 
                         fetches=['output_accuracy'], 
                         feed_dict={'images': B('images'), 'labels': B('labels')},
                         save_to=[V('accuracy')], mode='a')
)

In [ ]:
train_ppl = train_root + train_template
test_ppl = test_root + test_template

All parameter combinations we define through the dict where a key is a parameter name and value is a list of possible parameter values. Create a grid of parameters in the following way:

In [ ]:
grid = Option('layout', ['cna', 'can']) * Option('model', [VGG7, VGG16])

You can get all variants of config:

In [ ]:
configs = list(grid.gen_configs())
configs

Each element is a ConfigAlias. It's a Config dict of parameter values and dict with aliases for parameter values:

In [ ]:
configs[0].config(), configs[0].alias()

Note that in the first case `model` is a class but in the second `str`.

# Create Research object

Create an instance of `Research` class and add train and test pipelines and grid of parameters:

In [ ]:
research = (Research()
            .pipeline(train_ppl, variables='loss', name='train')
            .grid(grid))

Parameter ``name`` defines pipeline name inside ``research``. At each iteration train pipeline will be executed with ``.next_batch()`` and all ``variables`` from the pipeline will be saved so that variables should be added with ``mode='w'``.

Test pipeline will be executed with ``.run()`` at each 100 iterations because of parameters ``run=True``  and ``execute='%100'``. `execute` can be `int` (iteration), `str` (`%{step})`) or `list` of `int` and `str`. Pipeline variable ``accuracy`` will be saved after each execution. In order to add a mean value of accuracy on test dataset, you can define a function


In [ ]:
def get_accuracy(iteration, experiment, pipeline):
    import numpy as np
    pipeline = experiment[pipeline].pipeline
    acc = pipeline.get_variable('accuracy')
    return np.mean(acc)

research.function(get_accuracy, returns='accuracy', name='test_accuracy', execute='%100', pipeline='test')

That function will get iterartion, experiment and kwargs (in that case it's `pipeline='test'`").

Experiment is an OrderedDict for all pipelines and functions that were added to Research
and are running in current job. Key is a name of ExecutableUnit (class for function and pipeline), value is ExecutableUnit.
Each pipeline and function added to Research is saved as an ExecutableUnit. Each ExecutableUnit
has the following attributes:

    function : callable
        is None if ExecutableUnit is a pipeline
    pipeline : Pipeline
        is None if ExecutableUnit is a function
    root_pipeline : Pipeline
        is None if ExecutableUnit is a function or pipeline is not divided into root and branch
    result : dict
        current results of the ExecutableUnit. Keys are names of variables (for pipeline)
        or returns (for function) values are lists of variable values
    path : str
        path to the folder where results will be dumped
    exec_for : int, list of ints or None
    dump_for : int, list of ints or None
    to_run : bool
    variables : list
        variables (for pipeline) or returns (for function)
    on_root : bool
    args : list
    kwargs : dict()


# Run research

And now you can start research. At each iteration units will be exuted in the same order as they were added into Research (if unit must be executed).

In [ ]:
# research.run(n_reps=10, n_iters=1000, name='my_research', progress_bar=True)

... but don't hurry up if you have a lot of gpus because you can do research much more faster, just define `workers=4` and `gpu = [0, 1, 2, 3]` as a list of available devices. In that case you can run 4 jobs in parallel!

In [ ]:
# research.run(n_reps=10, n_iters=1000, workers=4, gpu=[0,1,2,3], name='my_research', progress_bar=True)

Moreover, if you have heavy loading you can do it just one time for few pipelines with models. In that case you can define research in the following way:

In [ ]:
research = (Research()
    .pipeline(root_pipeline=train_root, branch_pipeline=train_template, variables='loss', name='train')
    .pipeline(root_pipeline=test_root, branch_pipeline=test_template,
              variables='accuracy', name='test', run=True, execute='%100', import_from='train')
    .grid(grid)
    .function(get_accuracy, returns='accuracy', name='test_accuracy', execute='%100', pipeline='test')
)

In order to achieve parallelization in branches we added `device` into model_config.

And now you can define the number of branches in each worker:

In [ ]:
research.run(n_reps=2, n_iters=1000, workers=2, branches=2, gpu=[0,1,2,3], name='my_research', progress_bar=True)

At last iteration all results will be saved into `{research_name}/results/{config_alias}/{repetition_index}/{unitname}_{iteration}` add a dict pickled by dill.

In [ ]:
import dill

with open('my_research/results/layout_can-model_VGG16/0/train_1000', 'rb') as f:
    result_train = dill.load(f)

with open('my_research/results/layout_can-model_VGG16/0/test_accuracy_1000', 'rb') as f:
    result_test = dill.load(f)

print(result_train.keys()) 
print(result_test.keys())

In [ ]:
import matplotlib.pyplot as plt

plt.plot(result_train['iteration'], result_train['loss'])

In [ ]:
plt.plot(result_test['iteration'], result_test['accuracy'])
plt.ylim((0,1))
plt.xlim((0,1000))

# Dumping of results and logging

By default if unit has `varaibles` or `returns` then results will be dumped at last iteration. But there is unit parameter `dump` that allows to save result not only in the end. It defines as `execute` parameter. For example, dump train results each 200 iterations. Besides, each research has log file. In order to add information about unit execution and dumping into log, define `logging=True`.

In [ ]:
research = (Research()
    .pipeline(root_pipeline=train_root, branch_pipeline=train_template, variables='loss', name='train', dump='%200')
    .pipeline(root_pipeline=test_root, branch_pipeline=test_template,
              variables='accuracy', name='test', run=True, execute='%100', import_from='train', logging=True)
    .grid(grid)
    .function(get_accuracy, returns='accuracy', name='test_accuracy', execute='%100', pipeline='test')
)

In [ ]:
research.run(n_reps=2, n_iters=1000, workers=2, branches=2, gpu=[0,1,2,3], name='my_research', progress_bar=True)

# Functions on root

All functions and pipelines if `branches > 0` executed in parallel threads so sometime it can be a problem. In order to allow run function in main thread there exists parameter `on_root`. Function that will be added with `on_root=True` will get `iteration`, `experiments` and `kwargs`. `experiments` is a list of experiments that was defined above (`OrderedDict` of ExecutableUnits). Simple example of usage:

In [ ]:
def on_root(iteration, experiments):
    print("On root", iteration)

In [ ]:
research = (Research()
    .function(on_root, on_root=True, execute=10, logging=True)
    .pipeline(root_pipeline=train_root, branch_pipeline=train_template, variables='loss', name='train')
    .pipeline(root_pipeline=test_root, branch_pipeline=test_template,
              variables='accuracy', name='test', run=True, execute='%100', import_from='train', logging=True)
    .grid(grid)
    .function(get_accuracy, returns='accuracy', name='test_accuracy', execute='%100', pipeline='test')
)

That function will be executed just one time on 10 iteration and will be executed one time for all branches in task.

In [ ]:
research.run(n_reps=1, n_iters=100, workers=2, branches=2, gpu=[0,1,2,3], name='my_research', progress_bar=True)